In [ ]:
#Import packages

import numpy as np
import pandas as pd
from PIL import Image
import os
import path

import torch
import torchvision
from torch.utils import data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
def target_to_oh(target):
    NUM_CLASS = 5  # hard code here, can do partial
    one_hot = torch.eye(NUM_CLASS)[target]
    return one_hot

In [ ]:
transform_bunch = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.ToTensor()])
#                                       transforms.Normalize((0.5139, 0.2727, 0.0891), (0.1533, 0.0909, 0.0400))])

In [ ]:
ds = torchvision.datasets.ImageFolder("../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images", transform_bunch, target_transform = target_to_oh)

In [ ]:
for x,y in ds:
    print(y)
    break

In [ ]:
percent = 0.2

split = int(len(ds) * percent)

indices = list(range(len(ds)))

In [ ]:
split

In [ ]:
train_sampler = SubsetRandomSampler(indices[split:])

valid_sampler = SubsetRandomSampler(indices[:split])

In [ ]:
bs = 100

dl = data.DataLoader(ds, bs, sampler=train_sampler)

valid_dl = data.DataLoader(ds, bs, sampler=valid_sampler)

In [ ]:
xx = 0
yy = 0

for x,y in valid_dl:
    print(x.shape,y.shape)
    xx = x
    yy = y
    break

In [ ]:
yy

In [ ]:
#looks like something is off... Yup --> We can just transform it at the end
ds.class_to_idx

In [ ]:
#Now I'll be making the model! It'll take elements off of nn.Module
class cnnmaker(nn.Module):
    #Initialization will only take in input channels
    def __init__(self, input_channels):
        super().__init__()
        
        #I looked at what the resnet architechture looked like and just implmented the block types (not the residual part just yet)
        def block(in_chan):
            return nn.Sequential(nn.Conv2d(in_chan, in_chan * 2, 3, 2, 1, padding_mode = "reflect"),
                                nn.BatchNorm2d(in_chan*2),
                                nn.ReLU())
        
        self.model = nn.Sequential(block(input_channels),
                                  block(input_channels*2),
                                  block(input_channels*4),
                                  block(input_channels*8),
                                  block(input_channels*16))
        
        self.second_model = nn.Sequential(nn.Linear(4704, 500),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(500, 100),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(100, 5))
    #The forward pass when you call it   
    def forward(self, images):
        pre_proc = self.model(images)
#         print(pre_proc.shape)
        
        formatted = torch.reshape(pre_proc, (images.shape[0], -1))
        
        return self.second_model(formatted)

In [ ]:
model = cnnmaker(3)

In [ ]:
model

In [ ]:
#Look at how many parameters
total = 0
for param in model.parameters():
    if param.requires_grad:
        total += param.numel()

print(total)

In [ ]:
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
model = model.to('cuda')
loss_func = loss_func.to('cuda')

In [ ]:
def accuracy(preds, target):
    correct = (preds == target).float()
    accuracy = correct.sum() / len(correct)
    return accuracy

In [ ]:
def train():
    model.train()
    
    for image, label in dl:
        image, label = image.type(dtype=torch.cuda.FloatTensor), label.type(dtype=torch.cuda.FloatTensor)
        optimizer.zero_grad()
        prediction = model(image)
        
        loss = loss_func(prediction, label)
        
        loss.backward()
        
        optimizer.step()
    
    with torch.no_grad():
        total_iter = 0
        total_acc = 0
        for image, label in valid_dl: 
            image, label = image.to('cuda', dtype=torch.float), label.to('cuda', dtype=torch.float)
            prediction = model(image)
            total_acc += accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
            total_iter += 1
            if total_iter == 7:
                print("valid",total_acc/total_iter)
                total_iter = 0
                total_acc = 0
                for image, label in dl: 
                    image, label = image.to('cuda', dtype=torch.float), label.to('cuda', dtype=torch.float)
                    prediction = model(image)
                    total_acc += accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
                    total_iter += 1
                    if total_iter == 7:
                        print(total_acc/total_iter)
                        return "done"

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.003)

In [ ]:
for _ in range(4):
    train()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
for _ in range(10):
    train()

In [ ]:
for _ in range(20):
    train()

In [ ]:
for _ in range(20):
    train()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
for _ in range(20):
    train()

In [ ]:
xxx = 0
yyy = 0

for x,y in dl:
    print(x.shape,y.shape)
    xxx = x
    yyy = y
    break

In [ ]:
yyy = yyy.type(dtype=torch.cuda.FloatTensor)
torch.argmax(yyy, dim = 1).float()

In [ ]:
xxx = xxx.type(dtype=torch.cuda.FloatTensor)
prediction = model(xxx)
torch.argmax(prediction, dim = 1).float()


In [ ]:
accuracy(torch.argmax(prediction, dim = 1).float(),torch.argmax(yyy, dim = 1).float())

In [ ]:
with torch.no_grad():
    total_iter = 0
    total_acc = 0
    for image, label in dl: 
        image, label = image.to('cuda', dtype=torch.float), label.to('cuda', dtype=torch.float)
        prediction = model(image)
        h = accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
        print(h)
        total_acc += h
        total_iter += 1
        if total_iter == 20:
            print(total_acc/total_iter)